## Robust Facility Location Problem

In this demo, we will consider the problem of robust facility location, based on some (uncertain) projections about demand. 
Let's consider the following, where we plan to pick a subset of 10 facility locations with indices $i \in I$ to service 50 customers with indices $j \in J$. 
Each facility has a capacity $s_i$, and a construction cost of $f_i$. 
The transportation costs $c_{i,j}$ are defined across each facility-customer pair $\{i, j\}$, and are variable depending on the quantity of goods moved. 

We begin by launching a Julia environment, adding some packages and plotting utilities, and generating some random (but interesting) data. 

In [ ]:
# Packages
using Pkg
Pkg.activate(".")

using JuMP, GLPK, Random, LinearAlgebra, DataFrames, PyPlot
include("utils.jl")

n = 10 # Number of facilities
m = 50 # Number of customers

# Generating random data (please don't change the seeds.)
facilities = 0.6.*rand(MersenneTwister(5), n,2) .+ 0.2;
customers = rand(MersenneTwister(2), m, 2); 
c = [LinearAlgebra.norm(customers[i, :] .- facilities[j, :])[1] for j=1:n, i=1:m];
f = rand(MersenneTwister(3), n)*1 .+ 5;
s = rand(MersenneTwister(4), n)*2 .+ 15;
d = rand(MersenneTwister(5), m)*0.5 .+ 0.75;

At first, we will assume that the demand $d_j$ at each demand node is fixed and deterministic. This we call the nominal problem, stored in `nominal.jl`, which we can optimize to see the nominal facility locations under perfect knowledge. 

In [ ]:
include("nominal.jl")
model, x, y = facility_model(c, f)
optimize!(model)
plt = plot_solution(model, x, y)

However, the real world is not so straightforward. In reality, we expect some demand uncertainty, which we model as $d_j + (Pz)_j$, where $d_j$ is the nominal demand, and the $(Pz)_j$ is the demand perturbation at node $j$. $z_j$ describes a demand perturbation at any node $j$, which affects all nearby nodes through the $P$ matrix, within a distance of $R_D = 0.25$.


In [ ]:
R_D = 0.25
P = [0.2*exp(-1/R_D .*LinearAlgebra.norm(customers[i, :] .- customers[j, :])[1]) for j=1:m, i=1:m];
P = (P .>= 0.2*exp(-1/R_D .* R_D)) .* P
first(DataFrame(P, :auto), 5)

We will assume that the uncertain parameters $z$ come from within a budget uncertainty set, $||z||_{\infty} \leq \rho,~ ||z||_{1} \leq \Gamma$. The robust counterpart of the facility location problem can be found in `robust.jl`. For now, we will optimize the facility locations considering *static* transportation decisions, for $\rho = 1$ and $\Gamma = 5$. 

In [ ]:
# Robust model (3.2)
include("robust.jl")
ρ = 1
Γ = 5
model, x, y = robust_facility_model(c, f, ρ, Γ)
optimize!(model)
plt = plot_solution(model, x, y)

We can do a more in depth analysis of the **stability** of optimal facility locations by keeping the maximal demand perturbation $\rho$ constant, but changing the size of $\Gamma$ and reoptimizing. Running the cell below will allow us to compare the relative numbers and locations of our facilities as we change the size of our uncertainty set.  

In [ ]:
Γs = collect(0:8)
df = DataFrame("Gamma" => [], "f+c" => [], "f" => [], "c" => [], "nx" => [])
fig = figure("3x3_subplot", figsize = (15, 15))
for k = 1:length(Γs)
    Γ = Γs[k]
    model, x, y = robust_facility_model(c, f, ρ, Γ)
    optimize!(model)
    push!(df, Dict("Gamma" => Γ,
                     "f+c" => objective_value(model),   
                     "f" => value(sum(f[j] * x[j] for j = 1:n)),
                     "c" => value(sum(c[i, j] * y[i, j] for i=1:n, j=1:m)),
                     "nx" => sum(value.(x))))
    subplot(330 + k)
    plot_solution(model, x, y)
end

One thing to note is that there are three stable sets of facility locations as we increase the overall size of the uncertainty that $\Gamma$. We can also take a better look at the results by observing the table created in the for loop, which shows the magnitudes of the facility and transportation costs, as well as the number of facilities as we change $\Gamma$.  

In [ ]:
df